In [2]:
import json
import praw
import pandas as pd

In [4]:
with open('/home/gokul/.api_keys/reddit.keys') as auth_file:
    auth = json.loads(auth_file.read())

In [5]:
r = praw.Reddit(**auth)

In [6]:
df = pd.read_pickle('Data/working_sub.pkl')

In [7]:
link_ids = df.parent_id[df.parent_id.str.startswith('t3_')].unique()

In [8]:
del df

In [41]:
len(link_ids)

45400

In [3]:
columns = ['sub_id', 'created_utc', 'distinguished', 'edited',
           'locked', 'url', 'num_comments', 'over_18',
           'score', 'selftext', 'title', 'upvote_ratio',
           'downs', 'ups', 'is_original_content',
           'num_crossposts', 'spoiler', 'view_count']

In [43]:
def get_data(ids):
    df = pd.DataFrame(columns=columns)
    for i, sub_id in enumerate(ids):
        sm = r.submission(id=sub_id[3:])
        df.loc[i, 'sub_id'] = sub_id
        df.loc[i, 'created_utc'] = sm.created_utc
        df.loc[i, 'distinguished'] = sm.distinguished
        df.loc[i, 'edited'] = sm.edited
        df.loc[i, 'locked'] = sm.locked
        df.loc[i, 'url'] = sm.url
        df.loc[i, 'num_comments'] = sm.num_comments
        df.loc[i, 'over_18'] = sm.over_18
        df.loc[i, 'score'] = sm.score
        df.loc[i, 'selftext'] = sm.selftext
        df.loc[i, 'title'] = sm.title
        df.loc[i, 'upvote_ratio'] = sm.upvote_ratio
        df.loc[i, 'downs'] = sm.downs
        df.loc[i, 'ups'] = sm.ups
        df.loc[i, 'is_original_content'] = sm.is_original_content
        df.loc[i, 'num_crossposts'] = sm.num_crossposts
        df.loc[i, 'spoiler'] = sm.spoiler
        df.loc[i, 'view_count'] = sm.view_count
    return df

Scrapping in batches of thousands, so we have partial data if something fails.

In [52]:
for i in list(range(0, 45400, 1000)):
    ids = link_ids[i:i+1000]
    df = get_data(ids)
    df.to_pickle(f'Data/scrape/df_{i}.pkl')

In [4]:
df = pd.DataFrame(columns=columns)
for i in list(range(0, 45400, 1000)):
    df_t = pd.read_pickle(f'Data/scrape/df_{i}.pkl')
    df = pd.concat([df, df_t])

In [5]:
df.shape

(45400, 18)

In [6]:
df.reset_index(inplace=True, drop=True)

In [58]:
df.to_pickle('Data/scrape/df_final.pkl')